# Weighted Boxes Fusion (WBF) 是什么
当我们进行物体检测的时候，模型输出的目标检测框很多时候可能不止一个，这个时候我们需要选择其中的一个框作为最终输出，其他常见的算法还有NMS，soft-NMS。而WBF在许多情景下能比NMS 等算法有更好的表现。
![object detection](images/cat and dog.png)


In [ ]:
import numpy as np

先来制作一些假数据

In [ ]:
# x1,x2,y1,y2
boxes = [[2,2,4,4],[3,3,4,4],[5,5,6,6],[1,2,3,4],[2,3,5,7],[4,5,6,8]]
labels = [1,1,1,2,2,2]
scores = [0.1,0.8,0.9,0.2,0.3,0.6]

### 数据预处理
先将boxes 和 scores 按照 labels 的类型分好

In [ ]:
new_boxes = {}
new_scores ={}
new_labels =[]
for i in range(len(labels)):
  if labels[i] in new_boxes:
    new_boxes[labels[i]].append(boxes[i])
    new_scores[labels[i]].append(scores[i])
  else:
    new_boxes[labels[i]] = [boxes[i]]
    new_scores[labels[i]]=[scores[i]]
    new_labels.append(labels[i])
print(new_boxes)
print(new_scores)
print(new_labels)

{1: [[2, 2, 4, 4], [3, 3, 4, 4], [5, 5, 6, 6]], 2: [[1, 2, 3, 4], [2, 3, 5, 7], [4, 5, 6, 8]]}
{1: [0.1, 0.8, 0.9], 2: [0.2, 0.3, 0.6]}
[1, 2]


接下来我们要按照scores(confidence)的高低排序

In [ ]:
for i in new_labels:
  boxes = new_boxes[i]
  # 为了使用argmin,将scores转为numpy对象
  scores = np.array(new_scores[i])
  sorted_boxes =[]
  sorted_scores = []

  while len(scores) > 0:
    pos = np.argmax(scores)
    sorted_boxes.append(boxes[pos])
    sorted_scores.append(scores[pos])
    scores = np.delete(scores,pos)
  new_boxes[i] = sorted_boxes
  new_scores[i] = sorted_scores
print(new_boxes)
print(new_scores)

{1: [[5, 5, 6, 6], [3, 3, 4, 4], [2, 2, 4, 4]], 2: [[4, 5, 6, 8], [2, 3, 5, 7], [1, 2, 3, 4]]}
{1: [0.9, 0.8, 0.1], 2: [0.6, 0.3, 0.2]}


### WBF 主要算法
#### 生成L_box 和 Fusion 


1.   准备两个列表 Fusion, L_boxes
2.   遍历某个类中的所有box
3.   如果这个box和Fusion中所有的盒子IOU都没有超过 0.25:（论文中用的是0.55,这里为了方便用0.25)
  * 将这个box放入 L_boxes, 和Fusion最后面
4.   否则：
  * 将这个box放入L_boxes中与重复的Fusion同一个位置






首先先要写一个函数计算两个box的IoU

In [ ]:
def findoverlap(box,Fusion):
  area = []
  for _,f_box in Fusion.items():
    lower_bound = [max(box[0],f_box[0]),max(box[1],f_box[1])]
    upper_bound = [min(box[2],f_box[2]),min(box[3],f_box[3])]
    w = upper_bound[0] - lower_bound[0]
    h = upper_bound[1] - lower_bound[1]
    if w <  0 or h < 0:
      area.append(0)
    else:
      box_area = (box[2]-box[0])*(box[3]-box[1])
      f_box_area = (f_box[2]-f_box[0])*(f_box[3]-f_box[1])
      u_area = box_area + f_box_area - w*h
      area.append(w*h/u_area)
  return area

In [ ]:
# 测试一下
findoverlap(box  = [1,1,3,3],Fusion = {0:[2,2,3,3],1:[4,4,5,5]})

[0.25, 0]

In [ ]:
Fusion = {}
L_boxes ={}
L_scores ={}
num_pos = 0
# 先拿一个类做示范，后面可以改成一个循环
label = new_labels[0] 
for i, box in enumerate(new_boxes[label]):
  IoU = findoverlap(box,Fusion)
  if len(IoU) == 0 or sum(IoU) == 0 or max(IoU) < 0.25:
    # 没有重叠的
    Fusion[num_pos]=box
    L_boxes[num_pos] = [box]
    L_scores[num_pos] = [new_scores[label][i]]
    num_pos += 1
  else:
    pos = np.argmax(np.array(IoU))
    L_boxes[pos].append(box)
    L_scores[pos].append(new_scores[label][i])
print(Fusion)
print(L_boxes)
print(L_scores)

{0: [5, 5, 6, 6], 1: [3, 3, 4, 4]}
{0: [[5, 5, 6, 6]], 1: [[3, 3, 4, 4], [2, 2, 4, 4]]}
{0: [0.9], 1: [0.8, 0.1]}


#### 计算L_box中加权平均值
现在我们知道在L_boxes中每个位置，都会存在一个以上的box,但我们最后决定只留一个box。这时，我们会计算每个位置上boxes的加权平均值，也就是把它们和为一体。

In [ ]:
result_boxes = []
for i in range(num_pos):
  result_box = [0,0,0,0]
  for j, box in enumerate(L_boxes[i]):
    result_box[0] += box[0] * L_scores[i][j] /sum(L_scores[i])
    result_box[1] += box[1] * L_scores[i][j] /sum(L_scores[i])
    result_box[2] += box[2] * L_scores[i][j] /sum(L_scores[i])
    result_box[3] += box[3] * L_scores[i][j] /sum(L_scores[i])
  result_boxes.append(result_box)
print(result_boxes)

[[5.0, 5.0, 6.0, 6.0], [2.8888888888888893, 2.8888888888888893, 4.0, 4.0]]


#### 重新为每个box计算confident
现在我们已经得到了可以输出的，没有重叠的box，但是还是要为每个box计算一个confidence.
confidence可以用最大值，平均值等等

In [ ]:
result_scores = []
for i in range(num_pos):
  result_scores.append(sum(L_scores[i])/len(L_scores[i]))
print(result_scores)

[0.9, 0.45]


到此，已经计算完毕了

# K-folds情况
有的时候会让多个模型同时预测，前面的步骤几乎是一样的，但最后求confidence的时候，要降低box数量比较少的目标的confidence(很少的模型预测出了这个)，方法有两个
$$ C = C \times \frac{min(Nboxes,Nmodels)}{Nmodels}$$
or
$$C = C \times \frac{Nboxes}{Nmodels}$$

#### Reference
Weighted Boxes Fusion: ensembling boxes for object detection models
https://arxiv.org/abs/1910.13302